# Análisis de frecuencia en Embalses usando imagenes satelitales

## Dirección Nacional de Aguas (DINAGUA) - Ministerio de Ambiente

### Obtener la imagen Satelital del Joint Research Center's (JRC) Global Surface Water Mapping Layers, v1.2

Importar los modulos de Python necesarios

In [119]:
# Import the packages
import ee
import geemap
import os
import geemap.colormaps as cm

In [120]:
# Quitar el comentario para correr si no se esta inicializado y autenticado la app

# ee.Authenticate()
# ee.Initialize()

Generar el Mapa base centrado en Uruguay

In [121]:
Map = geemap.Map(center=[-32.7,-56], zoom=7)
Map

Map(center=[-32.7, -56], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=…

Para definir un área de interes, se debe realizar lo siguiente: 
1. Ir a la barra de digitalización ubicado en parte izquierda del mapa
2. Seleccionar una herramienta de digitalización. Por ejemplo, seleccionamos dibujar un rectangulo
3. Dibujamos el area de interes y cerramos el área. 

La siguiente linea de comando permite extraer el poligono determinado por el usuario y lo asigna a la variable roi 

In [122]:
# Toma la geometria que defina el usuario en el mapa
feature = Map.draw_last_feature
# Si el usuario no define una geometria, entonces define un poligono por defecto en Canelon Grande
if feature is None:
    geom = ee.Geometry.Polygon(
        [
            [
                [-56.30, -34.50],
                [-56.18, -34.50],
                [-56.18, -34.40],
                [-56.30, -34.40],
                [-56.30, -34.50],
            ]
        ]
    )
    feature = ee.Feature(geom, {})

roi = feature.geometry()

Buscamos una colección de imagenes de la Misión Copernicus Sentinel-2. Ademas se realiza lo siguiente:
1. Filtrar por area de interes (roi)
2. Seleccionar el producto "occurrence"

In [123]:
occurrence = ee.Image("JRC/GSW1_4/GlobalSurfaceWater") \
    .clip(roi) \
    .select('occurrence')

In [124]:
freq = 90

mask = occurrence.gt(freq).unmask(1)
water_mask = occurrence.updateMask(mask)

In [125]:
vis_occurrence = {'min' : 0, \
    'max' : 100, \
    'palette': ['red', 'blue']
    }

waterVisParam = {'palette': 'blue'}

Map.addLayer(occurrence.updateMask(occurrence.divide(100)),vis_occurrence, name="Water Occurrence (1984-2022)",shown = False)
Map.addLayer(water_mask,waterVisParam,name = 'Water 90% frequency',shown = True, opacity = 0.4)

### Comparando ocurrencia y situación actual

In [126]:
# Definimos una fecha de inicio y final para buscar imagenes Sentinel Disponibles
startDate = '2023-03-01'
endDate = '2023-03-28'
# Filtramos
Collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterBounds(roi) \
    .filterDate(startDate, endDate) \
    .filterMetadata("CLOUDY_PIXEL_PERCENTAGE","less_than",10)


Extraemos la imagen con menor porcentaje de nubes y extremos para el area de interes (roi). Esta imagen se agrega al mapa mostrado arriba

In [127]:
image = Collection.first().clip(roi)
# Parametros para graficar
Map.centerObject(roi)
vis_param = {'min': 228, 'max': 1705, 'bands': ['B4', 'B3', 'B2'], 'RGB': 1.5}
Map.addLayer(image, vis_param, "Color Verdadero",shown = False)
Map.addLayer(roi,{},'Área de Interes',shown = False)
# Mostramos la fecha en que se tomó la imagen satelital
date = ee.Date(image.get('system:time_start')).format('dd-MMM-YYYY')
print(f'La imagen corresponde al día {date.getInfo()}')

La imagen corresponde al día 06-Mar-2023


#### Calculamos indicadores

In [128]:
AWEI = image.expression('4*(GREEN - SWIR1) - (0.25*NIR + 2.75*SWIR2)', 
      {
      'GREEN': image.select('B3').multiply(0.0001),
      'NIR': image.select('B8').multiply(0.0001),
      'SWIR1': image.select('B11').multiply(0.0001),
      'SWIR2': image.select('B12').multiply(0.0001),
      })

# Normalized Difference Water Index (NDWI)
NDWI = image.normalizedDifference(['B3', 'B8'])

# Modified Normalized Difference Water Index (MNDWI)
MNDWI = image.normalizedDifference(["B8","B4"])

# Combinar los indicadores para obtener un indice compuesto
waterCombine = AWEI.lt(0).And(NDWI.lt(0)).And(MNDWI.gt(0)).rename('renameToSelectBands')

Generar una mascara para remover valores nulos

In [129]:
mask = waterCombine.eq(0)
water = waterCombine.updateMask(mask)

Agregar la capa de Agua disponible en la imagen

In [130]:
Map.centerObject(roi)
waterVisParam = {'palette':'red'}
Map.addLayer(water,waterVisParam, 'Agua Disponible',shown = True, opacity=0.7)
Map

Map(center=[-34.4500073323256, -56.239999999999995], controls=(WidgetControl(options=['position', 'transparent…

#### Descargar los GeoTiff para visualizar en QGIS

In [131]:
link_occurrence = occurrence.getDownloadURL(
    {
    'scale': 30,
    'region': roi,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    });



print(link_occurrence)


https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/170ae152127ccbc24fd80e8905400b05-7d682bb4548274c3a32b383b1e202300:getPixels


In [132]:
link_actual = water.getDownloadURL(
    {
    'scale': 10,
    'region': roi,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    });
print(link_actual)

https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/cb131eaafb93170d58728b9bf87fc988-089335edff361127b9a2aff7dd45f161:getPixels
